In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from collections import Counter
from functools import cmp_to_key
import json
import pickle as pkl
import os
print(os.listdir("../input"))
working_dir = "../input"
# Any results you write to the current directory are saved as output.


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import  OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report



from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout



['test_images', '.DS_Store', 'train_sentiment', 'move.py', 'train', 'labels.pkl', 'train_images']


Using TensorFlow backend.


> ## csv model

In [7]:
def load_csv(path):
    csv_dataset = pd.read_csv(path,index_col="PetID")
    return csv_dataset
csv_dataset = load_csv(working_dir+"/train/"+"train.csv")
print(csv_dataset.shape)
csv_dataset.head()

(14993, 23)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PhotoAmt,AdoptionSpeed
PetID,,,,,,,,,,,,,,,,,,,,,
86e1089a3,2,Nibble,3,299,0,1,1,7,0,1,...,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,2
6296e909a,2,No Name Yet,1,265,0,1,1,2,0,2,...,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,2.0,0
3422e4906,1,Brisco,1,307,0,1,2,7,0,2,...,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,7.0,3
5842f1ff5,1,Miko,4,307,0,2,1,2,0,2,...,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",8.0,2
850a43f90,1,Hunter,1,307,0,1,1,0,0,2,...,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,3.0,2


### we can see "Name" "Description" has NaN Value

In [8]:
csv_dataset.count()

Type             14993
Name             13736
Age              14993
Breed1           14993
Breed2           14993
Gender           14993
Color1           14993
Color2           14993
Color3           14993
MaturitySize     14993
FurLength        14993
Vaccinated       14993
Dewormed         14993
Sterilized       14993
Health           14993
Quantity         14993
Fee              14993
State            14993
RescuerID        14993
VideoAmt         14993
Description      14981
PhotoAmt         14993
AdoptionSpeed    14993
dtype: int64

In [10]:
print(csv_dataset["Name"].unique().shape)
count = Counter(csv_dataset["Name"].values.tolist())
sorted_count = sorted(count.items(), key = lambda x: x[1], reverse = True)
sorted_count

(9061,)


[(nan, 1257),
 ('Baby', 66),
 ('Lucky', 64),
 ('Brownie', 54),
 ('No Name', 54),
 ('Mimi', 52),
 ('Blackie', 49),
 ('Puppy', 45),
 ('Kitty', 39),
 ('Max', 39),
 ('Kittens', 39),
 ('Oreo', 36),
 ('Coco', 35),
 ('Tiger', 34),
 ('Angel', 33),
 ('Milo', 32),
 ('Bobby', 30),
 ('Snowy', 30),
 ('Lucy', 29),
 ('Kiki', 29),
 ('Lily', 28),
 ('Tom', 27),
 ('Ginger', 26),
 ('Puppies', 26),
 ('Prince', 25),
 ('Happy', 25),
 ('Daisy', 24),
 ('Kitten', 23),
 ('Simba', 23),
 ('Lola', 23),
 ('Bella', 23),
 ('Rocky', 23),
 ('No Name Yet', 22),
 ('Molly', 22),
 ('Momo', 22),
 ('Princess', 22),
 ('Luna', 22),
 ('Buddy', 21),
 ('Tommy', 21),
 ('Fluffy', 21),
 ('Panda', 21),
 ('Peanut', 20),
 ('Cookie', 20),
 ('Snow', 20),
 ('Bobo', 20),
 ('Toby', 19),
 ('Mickey', 18),
 ('Shadow', 18),
 ('Blacky', 18),
 ('Teddy', 17),
 ('Lulu', 17),
 ('Coffee', 17),
 ('Oyen', 17),
 ('Leo', 17),
 ('Minnie', 17),
 ('Bailey', 17),
 ('Pepper', 16),
 ('Boy', 16),
 ('Jojo', 16),
 ('Cutie', 16),
 ('Milky', 15),
 ('Mocha', 15),
 ('

## drop "RescuerID" and "Name"

In [11]:
csv_dataset["Name"].unique().shape, csv_dataset["RescuerID"].unique().shape

((9061,), (5595,))

In [12]:
csv_dataset.drop(["RescuerID","Name","Description"], axis = 1, inplace=True)

In [13]:
csv_dataset.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

## add nlp features

In [14]:
def get_max_abslute(sentences):
    max_ = 0
    real = 0
    for sentence in sentences:
        tmp = abs(sentence['sentiment']['score'])
        if tmp > max_:
            max_ = tmp
            real = sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
    return real


def get_min_abslute(sentences):
    min_ = float("inf")
    real = 0
    for sentence in sentences:
        tmp = abs(sentence['sentiment']['score'])
        if tmp < min_:
            min_ = tmp
            real = sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
    return real


def get_mean_abslute(sentences):
    sum_ = 0
    count = 0
    real = 0
    for sentence in sentences:
        sum_ += sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
        count += 1
    
    return sum_ / count
    

def func(value):
    if type(value) == type("value"):
        return 0
    else:
        return value
    
    
def get_sentiment_dict(path):
    score_max = {}
    score_min = {}
    score_mean = {}
    
    document_score = {}
    document_magnitude = {}

    
    
    for file in os.listdir(path):
        if file.endswith(".json"):
            PetID = file.split(".")[0]
            with open(path + file,"r") as f:
                content = json.loads(f.read())

            
            score_max[PetID] = get_max_abslute(content['sentences'])
            score_min[PetID] = get_min_abslute(content['sentences'])
            score_mean[PetID] = get_mean_abslute(content['sentences'])
            
            document_score[PetID] = content['documentSentiment']['score']
            document_magnitude[PetID] = content['documentSentiment']['magnitude']
    
    return score_max,score_min,score_mean,document_score,document_magnitude

In [15]:
train_nlp_path = working_dir + "/train_sentiment/"
score_max,score_min,score_mean,document_score,document_magnitude = get_sentiment_dict(train_nlp_path)


csv_dataset["score_max"] = csv_dataset.index
csv_dataset["score_max"] = csv_dataset["score_max"].replace(score_max)
csv_dataset["score_max"] = csv_dataset["score_max"].apply(func)

csv_dataset["score_min"] = csv_dataset.index
csv_dataset["score_min"] = csv_dataset["score_min"].replace(score_min)
csv_dataset["score_min"] = csv_dataset["score_min"].apply(func)


csv_dataset["score_mean"] = csv_dataset.index
csv_dataset["score_mean"] = csv_dataset["score_mean"].replace(score_mean)
csv_dataset["score_mean"] = csv_dataset["score_mean"].apply(func)


csv_dataset["document_score"] = csv_dataset.index
csv_dataset["document_score"] = csv_dataset["document_score"].replace(document_score)
csv_dataset["document_score"] = csv_dataset["document_score"].apply(func)


csv_dataset["document_magnitude"] = csv_dataset.index
csv_dataset["document_magnitude"] = csv_dataset["document_magnitude"].replace(document_magnitude)
csv_dataset["document_magnitude"] = csv_dataset["document_magnitude"].apply(func)

In [16]:
csv_dataset.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed', 'score_max', 'score_min', 'score_mean',
       'document_score', 'document_magnitude'],
      dtype='object')

### dummy category features and standard numerical features

1. category:  
    ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',  
    'Color3', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health',  
    'State','score_max', 'score_min', 'score_mean','document_score', 'document_magnitude']

2. numerical:
    ['Age','MaturitySize','FurLength', 'Quantity', 'Fee','VideoAmt', 'PhotoAmt']

In [17]:
numerical_features = ['Age','MaturitySize','FurLength', 'Quantity', 'Fee','VideoAmt', 'PhotoAmt','Breed1', 'Breed2','score_max', 'score_min', 'score_mean',
       'document_score', 'document_magnitude']
non_numerical_features = ['Type', 'Gender', 'Color1', 'Color2', 'Color3', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health','State']

scaler = StandardScaler()
csv_dataset.loc[:,numerical_features] = scaler.fit_transform(csv_dataset.loc[:,numerical_features])
csv_dataset = pd.get_dummies(csv_dataset, dummy_na=True, columns = non_numerical_features)

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [18]:
labels = csv_dataset["AdoptionSpeed"]
csv_dataset.drop(labels = ["AdoptionSpeed"], axis = 1, inplace=True)

labels.shape, csv_dataset.shape

((14993,), (14993, 75))

In [19]:
with open("labels.pkl","wb") as f:
    pkl.dump(labels,f)

with open("csv_dataset.pkl","wb") as f:
    pkl.dump(csv_dataset,f)

In [99]:
X_train, X_test, y_train, y_test = train_test_split(
    csv_dataset, labels, test_size=0.2, random_state=0)

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 5,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 6,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,           
    'subsample': 0.8,              # 随机采样训练样本
    'colsample_bytree': 0.8,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.01,                 
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}

clf = xgb.XGBModel(**params)
model = clf.fit(X_train,y_train)

/usr/local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [66]:
labels_encoder = OneHotEncoder()
one_hot_labels = labels_encoder.fit_transform(labels.values.reshape((labels.shape[0],1)))
one_hot_labels.shape

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(14993, 5)

In [68]:
model = Sequential()

model.add(Dense(units=100, input_dim= X_train.shape[1], activation = "relu"))
model.add(Dropout(rate = 0.3))
model.add(Dense(units=50,activation = "relu"))
model.add(Dropout(rate = 0.5))
model.add(Dense(units=20,activation = "relu"))
model.add(Dropout(rate = 0.3))
model.add(Dense(units=5))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', 
              optimizer= "adam",
              metrics=['accuracy'])

In [69]:
X_train, X_test, y_train, y_test = train_test_split(
    csv_dataset, one_hot_labels, test_size=0.3, random_state=0)

# class_weights = compute_class_weight('balanced',np.unique(labels),labels)
# sample_weights = np.array([class_weights[index[0,0]] for index in np.argmax(y_train,axis = 1)])
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=100, batch_size= 32)
#model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=50, batch_size= 256, sample_weight=sample_weights)

Train on 10495 samples, validate on 4498 samples
Epoch 1/100
10495/10495 [==============================] - 3s 248us/step - loss: 1.5351 - acc: 0.2637 - val_loss: 1.4488 - val_acc: 0.3281
Epoch 2/100
10495/10495 [==============================] - 1s 80us/step - loss: 1.4643 - acc: 0.3164 - val_loss: 1.4310 - val_acc: 0.3424
Epoch 3/100
10495/10495 [==============================] - 1s 81us/step - loss: 1.4399 - acc: 0.3304 - val_loss: 1.4111 - val_acc: 0.3555
Epoch 4/100
10495/10495 [==============================] - 1s 85us/step - loss: 1.4260 - acc: 0.3366 - val_loss: 1.4048 - val_acc: 0.3608
Epoch 5/100
10495/10495 [==============================] - 1s 86us/step - loss: 1.4154 - acc: 0.3517 - val_loss: 1.3999 - val_acc: 0.3595
Epoch 6/100
10495/10495 [==============================] - 1s 84us/step - loss: 1.4070 - acc: 0.3561 - val_loss: 1.3912 - val_acc: 0.3644
Epoch 7/100
10495/10495 [==============================] - 1s 85us/step - loss: 1.4040 - acc: 0.3593 - val_loss: 1.3881 - 